In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 12

# use this path to run on kaggle
# train_path = "../input/msbd5001-fall2019/train.csv"
# test_path = "../input/msbd5001-fall2019/test.csv"

# use this path to run on local
train_path = "../data/train.csv"
test_path = "../data/test.csv"

In [2]:
train = pd.read_csv(train_path)

train = train.drop(columns=['id'])
train

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...","Jul 2, 2018","10 Dec, 2013",372.0,96.0
1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...","Nov 26, 2016","12 Aug, 2015",23.0,0.0
2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...","Jul 2, 2018","28 Jan, 2014",3018.0,663.0
3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...","Nov 28, 2016","31 Mar, 2010",63078.0,1746.0
4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...","Mar 4, 2018","30 Jul, 2012",8841.0,523.0
5,2.933333,False,25800.0,RPG,"Single-player,Online Co-op,Local Co-op,Shared/...","RPG,Open World,Survival,Co-op,Fantasy,Online C...",NaN,"26 Mar, 2019",NaN,NaN
6,2.616667,False,3600.0,"Action,Indie","Single-player,Steam Achievements,Steam Trading...","Great Soundtrack,Action,Violent,Indie,Top-Down...","Feb 16, 2018","23 Oct, 2012",39776.0,1035.0
7,0.150000,True,0.0,Simulation,"Single-player,Multi-player,Online Multi-Player...","VR,Simulation,Utilities,Multiplayer","Jan 29, 2018","28 Apr, 2016",33.0,3.0
8,0.016667,False,4000.0,"Adventure,RPG","Single-player,Steam Achievements,Steam Trading...","RPG,Turn-Based,Adventure,Fantasy,Great Soundtr...","Nov 23, 2018","25 Feb, 2014",1875.0,39.0
9,0.016667,False,8800.0,"Action,Adventure","Single-player,Steam Achievements,SteamVR Colle...","Action,Adventure,VR,Shooter,Puzzle,FPS,First-P...","Dec 22, 2017","17 Oct, 2017",47.0,11.0


In [3]:
test_with_id = pd.read_csv(test_path)
test = test_with_id.drop(columns=['id'])

In [4]:
train['total_positive_reviews'] = train[['total_positive_reviews']].fillna(train[['total_positive_reviews']].mean())
test['total_positive_reviews'] = test[['total_positive_reviews']].fillna(test[['total_positive_reviews']].mean())

train['total_negative_reviews'] = train[['total_negative_reviews']].fillna(train[['total_negative_reviews']].mean())
test['total_negative_reviews'] = test[['total_negative_reviews']].fillna(test[['total_negative_reviews']].mean())

# fill with the mode
train['purchase_date'] = train['purchase_date'].fillna("Jun 27, 2019")
test['purchase_date'] = test['purchase_date'].fillna("Oct 25, 2017")


In [5]:
# feature engineering: calculate waiting days and own days
train["waiting_days"] = pd.to_datetime(train['purchase_date']) - pd.to_datetime(train['release_date'])
test["waiting_days"] = pd.to_datetime(test['purchase_date']) - pd.to_datetime(test['release_date'])
train["own_days"] =  pd.to_datetime('Dec 25, 2019')- pd.to_datetime(train['purchase_date'])
test["own_days"] = pd.to_datetime('Dec 25, 2019')- pd.to_datetime(test['purchase_date'])

train["waiting_days"] = pd.to_timedelta(train["waiting_days"]).dt.days
test["waiting_days"] = pd.to_timedelta(test["waiting_days"]).dt.days
train["own_days"] = pd.to_timedelta(train["own_days"]).dt.days
test["own_days"] = pd.to_timedelta(test["own_days"]).dt.days

# because of the mode to fillna, some days maybe illegal, set them to 0
train.waiting_days[train["waiting_days"] < 0] = 0
test.waiting_days[test["waiting_days"] < 0] = 0
train.waiting_days[train["own_days"] < 0] = 0
test.waiting_days[test["own_days"] < 0] = 0

In [6]:
train

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,waiting_days,own_days
0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...","Jul 2, 2018","10 Dec, 2013",372.000000,96.000000,1665,541
1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...","Nov 26, 2016","12 Aug, 2015",23.000000,0.000000,472,1124
2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...","Jul 2, 2018","28 Jan, 2014",3018.000000,663.000000,1616,541
3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...","Nov 28, 2016","31 Mar, 2010",63078.000000,1746.000000,2434,1122
4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...","Mar 4, 2018","30 Jul, 2012",8841.000000,523.000000,2043,661
5,2.933333,False,25800.0,RPG,"Single-player,Online Co-op,Local Co-op,Shared/...","RPG,Open World,Survival,Co-op,Fantasy,Online C...","Jun 27, 2019","26 Mar, 2019",15356.115493,3348.991549,93,181
6,2.616667,False,3600.0,"Action,Indie","Single-player,Steam Achievements,Steam Trading...","Great Soundtrack,Action,Violent,Indie,Top-Down...","Feb 16, 2018","23 Oct, 2012",39776.000000,1035.000000,1942,677
7,0.150000,True,0.0,Simulation,"Single-player,Multi-player,Online Multi-Player...","VR,Simulation,Utilities,Multiplayer","Jan 29, 2018","28 Apr, 2016",33.000000,3.000000,641,695
8,0.016667,False,4000.0,"Adventure,RPG","Single-player,Steam Achievements,Steam Trading...","RPG,Turn-Based,Adventure,Fantasy,Great Soundtr...","Nov 23, 2018","25 Feb, 2014",1875.000000,39.000000,1732,397
9,0.016667,False,8800.0,"Action,Adventure","Single-player,Steam Achievements,SteamVR Colle...","Action,Adventure,VR,Shooter,Puzzle,FPS,First-P...","Dec 22, 2017","17 Oct, 2017",47.000000,11.000000,66,733


In [7]:
# for bool
train['is_free'] = train['is_free'].astype(int)
test['is_free'] = test['is_free'].astype(int)

In [8]:
drop_columns = ['purchase_date', 'release_date']

train_rough = train.drop(columns=drop_columns)
test_rough = test.drop(columns=drop_columns)

In [9]:
train_rough.drop(train_rough.price.nlargest(2).index, inplace=True)

In [10]:
y = train_rough['playtime_forever']

from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train_rough, y, random_state = 0, test_size=0.2)

In [11]:
train_X

,playtime_forever,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,waiting_days,own_days
299,2.883333,0,8800.0,Strategy,"Single-player,Steam Cloud",Strategy,114.0,29.0,229,521
248,0.000000,0,13500.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Great Soundtrack,Spectac...",9066.0,562.0,2303,223
310,0.000000,0,7800.0,"Indie,RPG,Strategy","Single-player,Steam Achievements,Steam Trading...","Turn-Based Combat,Dark Fantasy,Dungeon Crawler...",41141.0,6291.0,1128,308
338,0.000000,0,5000.0,Adventure,"Single-player,Steam Achievements,Full controll...","Batman,Superhero,Story Rich,Choices Matter,Adv...",5034.0,1589.0,552,688
142,0.500000,0,800.0,"Casual,Indie,RPG,Simulation","Single-player,Steam Achievements,Steam Trading...","Simulation,Indie,Relaxing,Casual,Singleplayer,...",9318.0,1194.0,390,1565
21,1.616667,0,8800.0,"RPG,Strategy","Single-player,Multi-player,Online Multi-Player...","Strategy,4X,Turn-Based,Turn-Based Strategy,Fan...",8406.0,1780.0,802,1122
341,0.616667,1,0.0,"Free to Play,Indie","Single-player,Full controller support","Free to Play,Walking Simulator,First-Person,In...",42.0,32.0,186,1117
141,0.000000,0,11200.0,"Action,Adventure,Indie","Single-player,Steam Achievements,Steam Worksho...","Adventure,VR,Action,Puzzle,Indie,First-Person",365.0,55.0,187,612
49,0.400000,1,0.0,"Casual,Free to Play,Massively Multiplayer,Simu...","Single-player,Multi-player,Online Multi-Player...","Free to Play,VR,Casual,Simulation,Massively Mu...",14.0,4.0,595,769
311,0.000000,0,6800.0,"Action,Indie,RPG,Strategy","Single-player,Steam Achievements,Full controll...","Indie,Action,RPG,Rogue-lite,Strategy,Card Game...",363.0,67.0,1196,170


In [12]:
# split the words 
generes_list = train_X['genres'].str.split(',',expand=False).to_numpy()
categories_list = train_X['categories'].str.split(',',expand=False).to_numpy()
tags_list = train_X['tags'].str.split(',',expand=False).to_numpy()

generes_set = {item for i in generes_list for item in i}
categories_set = {item for i in categories_list for item in i}
tags_set = {item for i in tags_list for item in i}

generes_time_dict = {}
categories_time_dict = {}
tags_time_dict = {}

# generate the mean of time corresponding to different tags
for genere in generes_set:
    count = np.sum(train_X['genres'].str.contains(genere))
    generes_time_dict[genere] = np.sum(train_X[train_X['genres'].str.contains(genere)].playtime_forever) / count

for category in categories_set:
    count = np.sum(train_X['categories'].str.contains(category))
    categories_time_dict[category] = np.sum(train_X[train_X['categories'].str.contains(category)].playtime_forever) / count

for tag in tags_set:
    count = np.sum(train_X['tags'].str.contains(tag))
    tags_time_dict[tag] = np.sum(train_X[train_X['tags'].str.contains(tag)].playtime_forever) / count


In [13]:
def find_mean(current_dict, current_str):
    str_list = current_str.split(',')
    num_list = [0]
    for item in str_list:
        if item in current_dict:
            num_list.append(current_dict[item])
    return np.mean(num_list)

In [14]:
generes_time_dict

{'Racing': 1.4,
 'Animation & Modeling': 5.116666666666666,
 'Action': 3.7845959595959595,
 'Indie': 2.720723684210526,
 'Utilities': 5.116666666666666,
 'Sports': 1.2083333333333335,
 'Design & Illustration': 3.708333333333333,
 'Adventure': 4.446509009009009,
 'Free to Play': 0.7469696969696968,
 'Violent': 1.625,
 'Sexual Content': 3.25,
 'Casual': 0.6753086419753086,
 'Early Access': 0.6599999999999999,
 'Massively Multiplayer': 15.286666666666667,
 'Audio Production': 0.03333333333333333,
 'Gore': 0.0,
 'Nudity': 0.0,
 'Simulation': 3.933333333333333,
 'RPG': 4.116463414634146,
 'Strategy': 3.879166666666666}

In [15]:
train_X['genres_time_mean'] = train_X['genres'].apply(lambda x: find_mean(generes_time_dict, x))
train_X['categories_time_mean'] = train_X['categories'].apply(lambda x: find_mean(categories_time_dict, x))
train_X['tags_time_mean'] = train_X['tags'].apply(lambda x: find_mean(tags_time_dict, x))

val_X['genres_time_mean'] = val_X['genres'].apply(lambda x: find_mean(generes_time_dict, x))
val_X['categories_time_mean'] = val_X['categories'].apply(lambda x: find_mean(categories_time_dict, x))
val_X['tags_time_mean'] = val_X['tags'].apply(lambda x: find_mean(tags_time_dict, x))

test_X = test_rough
test_X['genres_time_mean'] = test_rough['genres'].apply(lambda x: find_mean(generes_time_dict, x))
test_X['categories_time_mean'] = test_rough['categories'].apply(lambda x: find_mean(categories_time_dict, x))
test_X['tags_time_mean'] = test_rough['tags'].apply(lambda x: find_mean(tags_time_dict, x))

In [16]:
train_X['categories_count'] = train_X['categories'].apply(lambda x: len(x.split(',')))
val_X['categories_count'] = val_X['categories'].apply(lambda x: len(x.split(',')))
test_X['categories_count'] = test_X['categories'].apply(lambda x: len(x.split(',')))

train_X['tags_count'] = train_X['tags'].apply(lambda x: len(x.split(',')))
val_X['tags_count'] = val_X['tags'].apply(lambda x: len(x.split(',')))
test_X['tags_count'] = test_X['tags'].apply(lambda x: len(x.split(',')))

train_X['genres_count'] = train_X['genres'].apply(lambda x: len(x.split(',')))
val_X['genres_count'] = val_X['genres'].apply(lambda x: len(x.split(',')))
test_X['genres_count'] = test_X['genres'].apply(lambda x: len(x.split(',')))

In [17]:
train_X = train_X.select_dtypes(exclude=['object'])
val_X = val_X.select_dtypes(exclude=['object'])
test_X = test_X.select_dtypes(exclude=['object'])

In [18]:
train_X.describe()

,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews,waiting_days,...,genres_time_mean,categories_time_mean,tags_time_mean,categories_count,tags_count,genres_count
count,284.000000,284.000000,284.000000,284.000000,284.00000,284.000000,...,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000
mean,3.174061,0.070423,6980.982394,13760.461674,3459.84152,1009.454225,...,2.513840,3.455590,3.284472,5.355634,17.316901,2.478873
std,12.060766,0.256309,5198.560988,40452.448590,27343.40263,885.365674,...,0.661033,1.206483,1.250608,2.807789,4.715765,1.181233
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.474093,0.788889,1.006025,1.000000,1.000000,1.000000
25%,0.000000,0.000000,3600.000000,787.750000,119.50000,360.250000,...,2.168440,2.505164,2.390412,4.000000,16.750000,2.000000
50%,0.050000,0.000000,6750.000000,2894.500000,414.00000,791.500000,...,2.596122,3.158477,3.089629,5.000000,20.000000,2.000000
75%,1.445833,0.000000,9000.000000,9403.250000,1124.50000,1472.000000,...,2.743702,4.281417,3.878106,7.000000,20.000000,3.000000
max,113.800000,1.000000,30800.000000,440902.000000,436046.00000,4320.000000,...,6.467710,6.998455,11.136712,19.000000,20.000000,8.000000


In [19]:
test_X.describe()

,is_free,price,total_positive_reviews,total_negative_reviews,waiting_days,own_days,genres_time_mean,categories_time_mean,tags_time_mean,categories_count,tags_count,genres_count
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.066667,6213.311111,11146.727273,1656.602273,998.966667,659.166667,2.395502,3.378558,3.184850,5.311111,17.311111,2.644444
std,0.250841,4860.918274,23965.791872,5656.019825,889.486955,242.933720,0.564424,1.194678,1.004746,2.846817,4.433388,1.173788
min,0.000000,0.000000,0.000000,0.000000,0.000000,151.000000,0.373485,1.453832,1.650463,1.000000,4.000000,1.000000
25%,0.000000,3600.000000,591.750000,86.000000,334.000000,526.500000,1.960635,2.587870,2.414228,4.000000,15.250000,2.000000
50%,0.000000,6150.000000,1918.000000,293.500000,686.000000,662.000000,2.389078,2.929546,3.030739,5.000000,20.000000,3.000000
75%,0.000000,7725.000000,9344.500000,1104.000000,1278.500000,785.000000,2.743702,4.285720,3.732712,7.000000,20.000000,3.000000
max,1.000000,26800.000000,149032.000000,51387.000000,3850.000000,1164.000000,4.970000,6.416109,7.394144,17.000000,20.000000,6.000000


In [20]:
train_X = train_X.drop(columns=['playtime_forever'])
val_X = val_X.drop(columns=['playtime_forever'])

In [21]:
# norm it
train_x_norm = (train_X - train_X.min()) / (train_X.max() - train_X.min())
val_x_norm = (val_X - val_X.min()) / (val_X.max() - val_X.min())
test_x_norm = (test_X - test_X.min()) / (test_X.max() - test_X.min())

In [22]:
test_X.describe()

,is_free,price,total_positive_reviews,total_negative_reviews,waiting_days,own_days,genres_time_mean,categories_time_mean,tags_time_mean,categories_count,tags_count,genres_count
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.066667,6213.311111,11146.727273,1656.602273,998.966667,659.166667,2.395502,3.378558,3.184850,5.311111,17.311111,2.644444
std,0.250841,4860.918274,23965.791872,5656.019825,889.486955,242.933720,0.564424,1.194678,1.004746,2.846817,4.433388,1.173788
min,0.000000,0.000000,0.000000,0.000000,0.000000,151.000000,0.373485,1.453832,1.650463,1.000000,4.000000,1.000000
25%,0.000000,3600.000000,591.750000,86.000000,334.000000,526.500000,1.960635,2.587870,2.414228,4.000000,15.250000,2.000000
50%,0.000000,6150.000000,1918.000000,293.500000,686.000000,662.000000,2.389078,2.929546,3.030739,5.000000,20.000000,3.000000
75%,0.000000,7725.000000,9344.500000,1104.000000,1278.500000,785.000000,2.743702,4.285720,3.732712,7.000000,20.000000,3.000000
max,1.000000,26800.000000,149032.000000,51387.000000,3850.000000,1164.000000,4.970000,6.416109,7.394144,17.000000,20.000000,6.000000


In [23]:
test_x_norm

,is_free,price,total_positive_reviews,total_negative_reviews,waiting_days,own_days,genres_time_mean,categories_time_mean,tags_time_mean,categories_count,tags_count,genres_count
0,0.0,0.130597,0.017493,0.021834,0.610390,0.268509,0.515655,0.240240,0.252689,0.0625,1.0000,0.2
1,0.0,0.432836,0.038663,0.043493,0.117403,0.006910,0.577412,0.600699,0.436327,0.3750,1.0000,0.4
2,0.0,0.078358,0.004610,0.002588,0.220260,0.010859,0.491638,0.192443,0.104885,0.1250,0.3750,0.4
3,0.0,0.134328,0.000450,0.000759,1.000000,0.536032,0.474512,0.609099,0.332955,0.1250,0.0000,0.2
4,0.0,0.126866,0.270707,0.072158,0.332987,0.871668,0.515655,0.572284,0.426676,0.3750,1.0000,0.2
5,0.0,0.253731,0.002597,0.001576,0.024675,0.413623,0.532455,0.296674,0.095655,0.2500,0.4375,0.4
6,0.0,0.253731,0.010924,0.007006,0.164935,0.047384,0.399971,0.805099,0.417551,0.6250,0.9375,0.6
7,0.0,0.067164,0.001718,0.001615,0.927013,0.485686,0.515655,0.000000,0.159129,0.0000,0.5000,0.2
8,0.0,0.253731,0.011897,0.025104,0.697662,0.404738,0.231019,0.781721,0.499302,0.6250,1.0000,0.4
9,0.0,0.111903,0.025981,0.019577,0.246234,0.983218,0.498579,0.672631,0.289365,0.5625,1.0000,0.2


In [24]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

knn_model = KNeighborsRegressor(n_neighbors=20, 
                                leaf_size=15, 
                                p=4)
knn_model.fit(train_x_norm, train_y)
knn_rough_preds = knn_model.predict(train_x_norm)
print(sqrt(mean_squared_error(train_y, knn_rough_preds)))
knn_rough_preds = knn_model.predict(val_x_norm)
print(sqrt(mean_squared_error(val_y, knn_rough_preds)))

10.321160297184802
7.750752849747672


In [25]:
test_rough_predict = knn_model.predict(test_x_norm)
output = pd.DataFrame(pd.DataFrame({
        "id": test_with_id.id,
        "playtime_forever": test_rough_predict
    }))


output.to_csv("knn_submission.csv", index = False)